*FEATURES:
1. Interactive Chat Interface (Gradio-based)
2. Multi-modal input (Image + Text)
3. Disease information database with treatment recommendations
4. Confidence scoring and uncertainty handling
5. Batch processing capabilities
6. Model explainability (Grad-CAM visualizations)
7. Conversation memory
8. Multi-language support structure
9. Voice input/output capabilities
10. Exportable reports*



In [1]:
!pip install -q tensorflow opencv-python gradio gTTS pyttsx3
!pip install -q tf-keras-vis matplotlib seaborn pandas numpy pillow
!pip install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip uninstall -y httpx websockets


Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1
Found existing installation: websockets 15.0.1
Uninstalling websockets-15.0.1:
  Successfully uninstalled websockets-15.0.1


In [3]:
!pip install "httpx>=0.28.1,<1.0.0" "websockets>=13.0.0,<15.1.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 27.6 MB/s eta 0:00:00


In [4]:
!pip install -U huggingface-hub langgraph-sdk openai firebase-admin langsmith mcp google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 62.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.16.0
    Uninstalling openai-2.16.0:
      Successfully uninstalled openai-2.16.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.8
    Uninstalling langsmith-0.6.8:
      Successfully uninstalled langsmith-0.6.8
  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.3.3
    Uninstalling langgraph-sdk-0

In [1]:
!pip check


ipython 7.34.0 requires jedi, which is not installed.


In [2]:
# Step 2: Import Libraries

import tensorflow as tf
import numpy as np
import cv2
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import os
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50, VGG16
from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.utils.scores import CategoricalScore

# For text-to-speech
try:
    from gtts import gTTS
    GTTS_AVAILABLE = True
except:
    GTTS_AVAILABLE = False

# For advanced chat capabilities
from transformers import pipeline

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU Available: {tf.config.list_physical_devices('GPU')}")

✅ TensorFlow version: 2.19.0
✅ GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Step 3: Define the Complete Disease Knowledge Base
# Based on your research paper's 38 classes with added treatment information

DISEASE_KNOWLEDGE_BASE = {
    'Apple___Apple_scab': {
        'description': 'A fungal disease causing olive-green to black spots on leaves and fruit.',
        'cause': 'Venturia inaequalis fungus',
        'symptoms': [
            'Olive-green to black spots on leaves',
            'Scab-like lesions on fruit',
            'Premature leaf drop',
            'Distorted fruit growth'
        ],
        'treatment': [
            'Apply fungicides early in spring (bud break)',
            'Remove fallen leaves to reduce overwintering spores',
            'Plant resistant varieties like Liberty or Freedom',
            'Ensure proper spacing for air circulation',
            'Apply sulfur-based fungicides every 7-10 days'
        ],
        'prevention': [
            'Rake and destroy fallen leaves in autumn',
            'Prune trees to improve air circulation',
            'Avoid overhead irrigation'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Neem oil, sulfur sprays, or copper fungicides'
    },
    'Apple___Black_rot': {
        'description': 'A fungal disease affecting fruit, leaves, and branches causing rot and cankers.',
        'cause': 'Botryosphaeria obtusa fungus',
        'symptoms': [
            'Purple spots on leaves (frog-eye spots)',
            'Rotten fruit with concentric rings',
            'Cankers on branches',
            'Fruit mummification'
        ],
        'treatment': [
            'Prune out cankers during dormant season',
            'Remove mummified fruit and dead wood',
            'Apply fungicides at petal fall and cover sprays',
            'Use captan or thiophanate-methyl fungicides'
        ],
        'prevention': [
            'Sanitation: remove all dead wood and mummified fruit',
            'Control insects that wound the fruit',
            'Avoid excessive nitrogen fertilization'
        ],
        'severity': 'High',
        'organic_treatment': 'Lime sulfur sprays, copper fungicides'
    },
    'Apple___Cedar_apple_rust': {
        'description': 'A fungal disease requiring two hosts: apple and cedar/juniper trees.',
        'cause': 'Gymnosporangium juniperi-virginianae fungus',
        'symptoms': [
            'Yellow-orange spots on leaves',
            'Spiky projections on spots (telia)',
            'Premature leaf drop',
            'Fruit infection with similar spots'
        ],
        'treatment': [
            'Remove nearby cedar/juniper trees if possible (within 1-2 miles)',
            'Apply fungicides starting at pink bud stage',
            'Use myclobutanil or triadimefon fungicides',
            'Repeat applications every 7-10 days'
        ],
        'prevention': [
            'Plant resistant varieties (Red Delicious, McIntosh)',
            'Remove galls from cedar trees before spring',
            'Maintain distance from cedar/juniper trees'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Sulfur sprays, neem oil (limited effectiveness)'
    },
    'Apple___healthy': {
        'description': 'The plant appears healthy with no disease symptoms detected.',
        'care_tips': [
            'Continue regular monitoring',
            'Maintain proper irrigation schedule',
            'Apply balanced fertilizer in spring',
            'Prune for shape and air circulation',
            'Monitor for early signs of pest/disease'
        ],
        'next_inspection': '7-14 days',
        'severity': 'None'
    },
    'Blueberry___healthy': {
        'description': 'Healthy blueberry plant with normal growth patterns.',
        'care_tips': [
            'Maintain acidic soil pH (4.5-5.5)',
            'Ensure adequate moisture, especially during fruiting',
            'Mulch with pine needles or sawdust',
            'Prune old canes after 4-5 years',
            'Protect from birds with netting'
        ],
        'next_inspection': '7-14 days',
        'severity': 'None'
    },
    'Cherry_(including_sour)___Powdery_mildew': {
        'description': 'A fungal disease causing white powdery coating on leaves and fruit.',
        'cause': 'Podosphaera clandestina fungus',
        'symptoms': [
            'White powdery patches on leaves',
            'Distorted, curled leaves',
            'Infected fruit may crack or drop',
            'Stunted shoot growth'
        ],
        'treatment': [
            'Apply sulfur or potassium bicarbonate sprays',
            'Use fungicides like myclobutanil or fenbuconazole',
            'Begin treatments at first sign of disease',
            'Improve air circulation through pruning'
        ],
        'prevention': [
            'Plant resistant varieties',
            'Avoid excessive nitrogen',
            'Space plantings for good air flow',
            'Prune interior branches'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Milk spray (1:9 milk:water), neem oil, sulfur'
    },
    'Cherry_(including_sour)___healthy': {
        'description': 'Healthy cherry tree showing normal growth and fruit development.',
        'care_tips': [
            'Prune during dormant season',
            'Monitor for bacterial canker and brown rot',
            'Apply dormant spray for pest control',
            'Thin fruit if crop is heavy',
            'Harvest when fully ripe'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': {
        'description': 'A fungal disease causing rectangular gray lesions on leaves.',
        'cause': 'Cercospora zeae-maydis fungus',
        'symptoms': [
            'Small tan lesions that expand into rectangles',
            'Gray to brown spots with yellow halos',
            'Lesions parallel to leaf veins',
            'Severe defoliation from top down'
        ],
        'treatment': [
            'Apply foliar fungicides (azoxystrobin, propiconazole)',
            'Treat when disease reaches ear leaf or above',
            'Rotate crops (avoid corn after corn)',
            'Till residue to promote decomposition'
        ],
        'prevention': [
            'Plant resistant hybrids',
            'Rotate with soybeans or other non-host crops',
            'Manage residue through tillage',
            'Avoid continuous corn planting'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper fungicides (limited effectiveness), crop rotation'
    },
    'Corn_(maize)___Common_rust_': {
        'description': 'A fungal disease causing reddish-brown pustules on leaves.',
        'cause': 'Puccinia sorghi fungus',
        'symptoms': [
            'Small cinnamon-brown pustules on both leaf surfaces',
            'Pustules break open to release spores',
            'Severe infection causes yellowing and death',
            'Most severe on lower leaves first'
        ],
        'treatment': [
            'Apply fungicides if disease appears before tasseling',
            'Use products containing azoxystrobin or pyraclostrobin',
            'Treat when 10% of plants show pustules on upper leaves',
            'Fungicide application rarely economical for common rust'
        ],
        'prevention': [
            'Plant resistant hybrids',
            'Early planting to avoid peak rust season',
            'Scout fields regularly'
        ],
        'severity': 'Medium',
        'organic_treatment': 'None effective; rely on resistant varieties'
    },
    'Corn_(maize)___Northern_Leaf_Blight': {
        'description': 'A fungal disease causing cigar-shaped lesions on leaves.',
        'cause': 'Exserohilum turcicum fungus',
        'symptoms': [
            'Long, elliptical, gray-green lesions',
            'Lesions 1-6 inches long',
            'Dark gray spore masses in humid conditions',
            'Lesions start on lower leaves'
        ],
        'treatment': [
            'Apply fungicides if lesions present on upper leaves before tasseling',
            'Use strobilurin or triazole fungicides',
            'Economic threshold: 50% plants with lesions on ear leaf or above',
            'Rotate crops and manage residue'
        ],
        'prevention': [
            'Plant resistant hybrids with Ht genes',
            'Rotate crops (2-year minimum)',
            'Bury residue with tillage',
            'Avoid overhead irrigation'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper fungicides, proper spacing for air flow'
    },
    'Corn_(maize)___healthy': {
        'description': 'Healthy corn plant with normal growth and development.',
        'care_tips': [
            'Maintain adequate nitrogen levels',
            'Irrigate during critical stages (tasseling, silking)',
            'Scout for pests and diseases weekly',
            'Ensure proper plant population',
            'Side-dress nitrogen when plants are knee-high'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Grape___Black_rot': {
        'description': 'A serious fungal disease affecting grapes, causing fruit rot and leaf spots.',
        'cause': 'Guignardia bidwellii fungus',
        'symptoms': [
            'Brown circular spots on leaves with black dots',
            'V-shaped lesions on leaf margins',
            'Shriveled, black, mummified berries',
            'Young shoots with dark lesions'
        ],
        'treatment': [
            'Apply fungicides starting at bud break',
            'Continue every 7-14 days until harvest',
            'Use mancozeb, myclobutanil, or captan',
            'Remove mummified berries from vines and ground'
        ],
        'prevention': [
            'Practice strict sanitation (remove all mummies)',
            'Prune for air circulation',
            'Apply fungicides preventatively',
            'Select less susceptible varieties'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper sprays, sulfur, lime sulfur'
    },
    'Grape___Esca_(Black_Measles)': {
        'description': 'A complex fungal disease causing leaf striping and berry rot.',
        'cause': 'Multiple fungi (Phaeomoniella chlamydospora, etc.)',
        'symptoms': [
            'Interveinal chlorosis and necrosis (tiger stripes)',
            'Small, dark berries that shrivel',
            'Woody cankers on cordons and trunks',
            'Sudden wilting and death of arms'
        ],
        'treatment': [
            'No effective chemical treatment available',
            'Prune out infected wood during dormancy',
            'Avoid large pruning wounds',
            'Protect pruning wounds with fungicide paint',
            'Remove severely infected vines'
        ],
        'prevention': [
            'Avoid stress (drought, nutrient deficiency)',
            'Protect all pruning cuts immediately',
            'Use clean pruning tools',
            'Plant certified disease-free material'
        ],
        'severity': 'High',
        'organic_treatment': 'Prevention only; no effective organic treatment'
    },
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': {
        'description': 'A fungal disease causing reddish-brown spots on leaves.',
        'cause': 'Pseudocercospora vitis fungus',
        'symptoms': [
            'Circular to angular reddish-brown spots',
            'Spots may have yellow halos',
            'Severe defoliation in wet seasons',
            'Spots coalesce in severe infections'
        ],
        'treatment': [
            'Apply copper-based fungicides',
            'Use mancozeb or chlorothalonil',
            'Begin sprays at early bloom',
            'Continue every 10-14 days'
        ],
        'prevention': [
            'Promote air circulation through pruning',
            'Remove infected leaves',
            'Avoid overhead irrigation',
            'Plant in sunny locations'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, neem oil'
    },
    'Grape___healthy': {
        'description': 'Healthy grapevine showing normal growth and fruit development.',
        'care_tips': [
            'Maintain proper canopy management',
            'Monitor for powdery mildew, downy mildew, and black rot',
            'Apply preventative fungicide program',
            'Irrigate deeply but infrequently',
            'Test soil and fertilize accordingly'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Orange___Haunglongbing_(Citrus_greening)': {
        'description': 'A devastating bacterial disease spread by psyllid insects.',
        'cause': 'Candidatus Liberibacter asiaticus bacterium',
        'symptoms': [
            'Blotchy mottling of leaves (asymmetrical)',
            'Twisted, misshapen new growth',
            'Bitter, misshapen fruit with aborted seeds',
            'Yellow shoots (blonding)'
        ],
        'treatment': [
            'NO CURE AVAILABLE - Remove infected trees immediately',
            'Control psyllid vectors with insecticides',
            'Use systemic insecticides (imidacloprid) and foliar sprays',
            'Plant only certified disease-free nursery stock',
            'Consider tree removal to protect grove'
        ],
        'prevention': [
            'Control Asian citrus psyllid aggressively',
            'Use certified disease-free budwood',
            'Remove infected trees immediately',
            'Screen houses for nursery production',
            'Area-wide psyllid management programs'
        ],
        'severity': 'Critical',
        'organic_treatment': 'None - remove infected trees immediately'
    },
    'Peach___Bacterial_spot': {
        'description': 'A bacterial disease causing spots on leaves, fruit, and twigs.',
        'cause': 'Xanthomonas arboricola pv. pruni bacterium',
        'symptoms': [
            'Purple-black spots on leaves',
            'Shot holes in leaves after spot drop',
            'Deep pits on fruit with cracking',
            'Cankers on twigs with gumming'
        ],
        'treatment': [
            'Apply copper sprays during dormancy',
            'Use oxytetracycline sprays during growing season',
            'Begin applications at petal fall',
            'Continue every 10-14 days during wet periods'
        ],
        'prevention': [
            'Plant resistant varieties (Redhaven, Cresthaven)',
            'Avoid overhead irrigation',
            'Prune during dry weather',
            'Remove infected twigs during dormancy'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper sprays, streptomycin (where legal)'
    },
    'Peach___healthy': {
        'description': 'Healthy peach tree with normal fruit development.',
        'care_tips': [
            'Apply dormant copper spray for bacterial spot',
            'Thin fruit to 4-6 inches apart',
            'Prune for open center shape',
            'Monitor for plum curculio and Oriental fruit moth',
            'Harvest when fruit yields to gentle pressure'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Pepper,_bell___Bacterial_spot': {
        'description': 'A bacterial disease causing leaf spots and fruit lesions.',
        'cause': 'Xanthomonas euvesicatoria bacterium',
        'symptoms': [
            'Small water-soaked spots on leaves',
            'Spots turn brown with yellow halos',
            'Raised scabby spots on fruit',
            'Defoliation in severe cases'
        ],
        'treatment': [
            'Apply copper-based bactericides',
            'Use streptomycin sulfate (check local regulations)',
            'Begin at first sign of disease',
            'Rotate with non-host crops for 2-3 years',
            'Remove and destroy infected plants'
        ],
        'prevention': [
            'Use disease-free seed and transplants',
            'Avoid overhead irrigation',
            'Space plants for air circulation',
            'Remove crop debris after harvest',
            'Rotate crops'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper sprays, neem oil (suppression only)'
    },
    'Pepper,_bell___healthy': {
        'description': 'Healthy bell pepper plant with normal fruit production.',
        'care_tips': [
            'Maintain consistent soil moisture',
            'Fertilize with balanced NPK',
            'Stake or cage plants for support',
            'Monitor for aphids and thrips',
            'Harvest when fruits reach full size and color'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Potato___Early_blight': {
        'description': 'A fungal disease causing concentric ring spots on leaves.',
        'cause': 'Alternaria solani fungus',
        'symptoms': [
            'Dark brown spots with concentric rings (target spots)',
            'Yellowing of tissue around spots',
            'Spots start on older, lower leaves',
            'Stem lesions near soil line'
        ],
        'treatment': [
            'Apply chlorothalonil or mancozeb fungicides',
            'Use azoxystrobin or pyraclostrobin for curative control',
            'Begin sprays when plants are 12-18 inches tall',
            'Continue every 7-10 days',
            'Maintain adequate fertility (especially nitrogen)'
        ],
        'prevention': [
            'Rotate crops (3-4 year rotation)',
            'Remove volunteer potatoes',
            'Use certified seed potatoes',
            'Hill soil around stems',
            'Irrigate to avoid plant stress'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, Bacillus subtilis products'
    },
    'Potato___Late_blight': {
        'description': 'A devastating fungal-like disease causing rapid plant death.',
        'cause': 'Phytophthora infestans oomycete',
        'symptoms': [
            'Water-soaked dark lesions on leaves',
            'White fuzzy growth on leaf undersides in humid conditions',
            'Brown discoloration of tubers',
            'Rapid wilting and death of foliage'
        ],
        'treatment': [
            'Apply fungicides immediately when disease detected',
            'Use mancozeb, chlorothalonil, or cymoxanil',
            'For organic: copper hydroxide',
            'Destroy infected plants immediately',
            'Harvest tubers immediately if plants are infected'
        ],
        'prevention': [
            'Plant resistant varieties',
            'Use certified seed potatoes',
            'Hill soil to protect tubers',
            'Avoid overhead irrigation',
            'Apply preventative fungicides before disease appears',
            'Monitor weather (disease weather maps)'
        ],
        'severity': 'Critical',
        'organic_treatment': 'Copper fungicides (must be applied preventatively)'
    },
    'Potato___healthy': {
        'description': 'Healthy potato plant with normal tuber development.',
        'care_tips': [
            'Hill soil 2-3 times during growing season',
            'Maintain consistent soil moisture',
            'Scout for Colorado potato beetle and aphids',
            'Apply preventative fungicides for late blight',
            'Harvest when vines die back naturally'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Raspberry___healthy': {
        'description': 'Healthy raspberry plant with normal cane growth.',
        'care_tips': [
            'Prune out spent floricanes after harvest',
            'Thin primocanes to 4-5 per foot',
            'Maintain trellis support',
            'Monitor for spotted wing drosophila',
            'Apply mulch for weed control and moisture retention'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Soybean___healthy': {
        'description': 'Healthy soybean plant with normal pod development.',
        'care_tips': [
            'Monitor for aphids and spider mites',
            'Scout for white mold and SDS',
            'Ensure adequate nodulation for nitrogen fixation',
            'Irrigate during pod fill if dry',
            'Harvest at 13-15% moisture'
        ],
        'next_inspection': '7-14 days',
        'severity': 'None'
    },
    'Squash___Powdery_mildew': {
        'description': 'A fungal disease causing white powdery coating on leaves.',
        'cause': 'Podosphaera xanthii or Erysiphe cichoracearum fungi',
        'symptoms': [
            'White powdery patches on upper leaf surfaces',
            'Leaves turn yellow then brown',
            'Stunted growth and reduced yields',
            'Premature leaf death'
        ],
        'treatment': [
            'Apply sulfur, potassium bicarbonate, or neem oil',
            'Use synthetic fungicides (myclobutanil, propiconazole)',
            'Begin at first sign of disease',
            'Treat both upper and lower leaf surfaces',
            'Continue every 7-10 days'
        ],
        'prevention': [
            'Plant resistant varieties',
            'Ensure full sun exposure',
            'Space plants for air circulation',
            'Avoid overhead irrigation',
            'Remove severely infected leaves'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Milk spray (1:9 ratio), neem oil, sulfur, potassium bicarbonate'
    },
    'Strawberry___Leaf_scorch': {
        'description': 'A fungal disease causing purple spots and scorched leaf appearance.',
        'cause': 'Diplocarpon earlianum fungus',
        'symptoms': [
            'Small dark purple spots on leaves',
            'Spots enlarge with reddish-purple borders',
            'Leaf edges appear scorched or burned',
            'Severe defoliation in wet years'
        ],
        'treatment': [
            'Apply captan or thiram fungicides',
            'Use iprodione or fludioxonil for severe cases',
            'Begin applications at bloom',
            'Continue every 7-10 days',
            'Renovate beds after harvest'
        ],
        'prevention': [
            'Plant resistant varieties',
            'Use certified disease-free plants',
            'Avoid overhead irrigation',
            'Space rows for air circulation',
            'Mulch to prevent soil splash'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, neem oil'
    },
    'Strawberry___healthy': {
        'description': 'Healthy strawberry plant with normal fruit production.',
        'care_tips': [
            'Renovate beds after harvest (mow leaves, thin plants)',
            'Fertilize after renovation',
            'Maintain mulch between rows',
            'Monitor for spider mites and slugs',
            'Overwinter with straw mulch in cold climates'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    },
    'Tomato___Bacterial_spot': {
        'description': 'A bacterial disease causing spots on leaves, stems, and fruit.',
        'cause': 'Xanthomonas perforans or X. euvesicatoria bacteria',
        'symptoms': [
            'Small water-soaked spots on leaves',
            'Spots turn brown with yellow halos',
            'Severe leaf spotting and defoliation',
            'Scabby spots on fruit'
        ],
        'treatment': [
            'Apply copper-based bactericides',
            'Use streptomycin (where permitted)',
            'Begin at transplanting or first sign',
            'Rotate crops 2-3 years',
            'Avoid working in wet fields'
        ],
        'prevention': [
            'Use certified disease-free transplants',
            'Avoid overhead irrigation',
            'Space plants for air circulation',
            'Remove infected plant debris',
            'Rotate with non-host crops'
        ],
        'severity': 'High',
        'organic_treatment': 'Copper sprays (limited effectiveness)'
    },
    'Tomato___Early_blight': {
        'description': 'A fungal disease causing target-like spots and stem cankers.',
        'cause': 'Alternaria solani fungus',
        'symptoms': [
            'Dark brown spots with concentric rings (target board)',
            'Yellowing of older leaves',
            'Stem lesions near soil line',
            'Fruit rot at stem end'
        ],
        'treatment': [
            'Apply chlorothalonil or mancozeb',
            'Use azoxystrobin for curative control',
            'Begin when lower leaves first show spots',
            'Continue every 7-10 days',
            'Stake plants to improve air circulation'
        ],
        'prevention': [
            'Rotate crops 3 years',
            'Remove lower infected leaves',
            'Mulch to prevent soil splash',
            'Stake or cage plants',
            'Use resistant varieties (Mountain Magic)'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, Bacillus subtilis, neem oil'
    },
    'Tomato___Late_blight': {
        'description': 'A devastating disease causing rapid plant death (caused Irish Potato Famine).',
        'cause': 'Phytophthora infestans oomycete',
        'symptoms': [
            'Water-soaked dark lesions on leaves',
            'Brown lesions on stems',
            'White fungal growth on undersides in humid weather',
            'Firm, brown, sunken fruit lesions'
        ],
        'treatment': [
            'Apply fungicides immediately (preventative programs best)',
            'Use chlorothalonil, mancozeb, or cymoxanil',
            'For organic: copper hydroxide (must be preventative)',
            'Destroy infected plants immediately (bag and trash)',
            'DO NOT compost infected material'
        ],
        'prevention': [
            'Plant resistant varieties (Defiant, Iron Lady)',
            'Space plants for air circulation',
            'Avoid overhead irrigation',
            'Remove volunteer tomatoes and potatoes',
            'Apply preventative fungicides before disease appears',
            'Monitor weather conditions (blight weather)'
        ],
        'severity': 'Critical',
        'organic_treatment': 'Copper hydroxide (preventative only), destroy infected plants'
    },
    'Tomato___Leaf_Mold': {
        'description': 'A fungal disease causing yellow spots and olive-green mold on leaf undersides.',
        'cause': 'Passalora fulva (Cladosporium fulvum) fungus',
        'symptoms': [
            'Yellow spots on upper leaf surfaces',
            'Olive-green to brown mold on leaf undersides',
            'Leaves curl, wither, and drop',
            'Defoliation starts from lower leaves'
        ],
        'treatment': [
            'Apply chlorothalonil or mancozeb',
            'Use strobilurin fungicides (azoxystrobin)',
            'Improve greenhouse ventilation',
            'Remove and destroy infected leaves',
            'Reduce humidity around plants'
        ],
        'prevention': [
            'Plant resistant varieties (cf genes)',
            'Ensure excellent air circulation',
            'Avoid wetting foliage',
            'Space plants properly',
            'Control humidity in greenhouses'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, improve ventilation, reduce humidity'
    },
    'Tomato___Septoria_leaf_spot': {
        'description': 'A fungal disease causing small circular spots with dark borders.',
        'cause': 'Septoria lycopersici fungus',
        'symptoms': [
            'Small circular spots with gray centers and dark borders',
            'Tiny black dots (pycnidia) in center of spots',
            'Yellowing and browning of leaves',
            'Defoliation from bottom up'
        ],
        'treatment': [
            'Apply chlorothalonil or mancozeb',
            'Use azoxystrobin or pyraclostrobin',
            'Begin at first sign of disease',
            'Continue every 7-10 days',
            'Remove infected lower leaves'
        ],
        'prevention': [
            'Rotate crops 2-3 years',
            'Mulch to prevent soil splash',
            'Stake plants to improve air flow',
            'Remove infected debris in fall',
            'Avoid overhead irrigation'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, neem oil, remove infected leaves'
    },
    'Tomato___Spider_mites Two-spotted_spider_mite': {
        'description': 'Tiny arachnids causing stippling and webbing on leaves.',
        'cause': 'Tetranychus urticae mites',
        'symptoms': [
            'Tiny yellow or white spots (stippling) on leaves',
            'Fine webbing on undersides of leaves',
            'Leaves turn bronze or yellow and drop',
            'Tiny moving dots visible with hand lens'
        ],
        'treatment': [
            'Apply miticides (abamectin, bifenazate)',
            'Use insecticidal soap or horticultural oil',
            'Spray undersides of leaves thoroughly',
            'Repeat applications every 5-7 days',
            'Rotate miticide classes to prevent resistance'
        ],
        'prevention': [
            'Maintain adequate moisture (mites prefer dry conditions)',
            'Remove weeds that host mites',
            'Use reflective mulches to repel mites',
            'Introduce predatory mites',
            'Avoid broad-spectrum insecticides that kill predators'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Insecticidal soap, neem oil, horticultural oil, predatory mites'
    },
    'Tomato___Target_Spot': {
        'description': 'A fungal disease causing target-like spots on leaves and fruit.',
        'cause': 'Corynespora cassiicola fungus',
        'symptoms': [
            'Dark brown spots with concentric rings (target-like)',
            'Spots on leaves, stems, and fruit',
            'Fruit spots are sunken and dark',
            'Defoliation in severe cases'
        ],
        'treatment': [
            'Apply chlorothalonil or mancozeb',
            'Use strobilurin fungicides',
            'Begin at first sign of disease',
            'Continue every 7-10 days',
            'Remove infected plant debris'
        ],
        'prevention': [
            'Rotate crops with non-hosts',
            'Remove crop residue after harvest',
            'Space plants for air circulation',
            'Avoid overhead irrigation',
            'Stake plants'
        ],
        'severity': 'Medium',
        'organic_treatment': 'Copper fungicides, neem oil, Bacillus subtilis'
    },
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus': {
        'description': 'A devastating viral disease spread by whiteflies.',
        'cause': 'Tomato yellow leaf curl virus (TYLCV)',
        'symptoms': [
            'Small leaves that curl upward',
            'Yellowing of leaf margins',
            'Stunted, bushy plant growth',
            'Flower drop and no fruit set'
        ],
        'treatment': [
            'NO CURE - Remove and destroy infected plants immediately',
            'Control whitefly vectors with insecticides',
            'Use imidacloprid or thiamethoxam for whiteflies',
            'Use reflective mulches to repel whiteflies',
            'Remove weeds that host whiteflies'
        ],
        'prevention': [
            'Plant resistant varieties (Ty-1, Ty-2 genes)',
            'Use virus-free transplants',
            'Control whiteflies from transplanting',
            'Use row covers in high-risk periods',
            'Remove infected plants immediately',
            'Manage whitefly populations aggressively'
        ],
        'severity': 'Critical',
        'organic_treatment': 'None - remove infected plants; use resistant varieties and row covers'
    },
    'Tomato___Tomato_mosaic_virus': {
        'description': 'A viral disease causing mottled leaves and stunted growth.',
        'cause': 'Tobacco mosaic virus (TMV) and Tomato mosaic virus (ToMV)',
        'symptoms': [
            'Mottled light and dark green leaf patterns',
            'Distorted, fern-like leaves',
            'Stunted growth',
            'Brown streaks on stems (internal)'
        ],
        'treatment': [
            'NO CURE - Remove and destroy infected plants',
            'Control aphids and other vectors',
            'Wash hands and tools thoroughly',
            'Do not smoke near plants (tobacco can carry virus)',
            'Remove weeds that may host virus'
        ],
        'prevention': [
            'Plant resistant varieties (Tm-2 genes)',
            'Use certified disease-free seed',
            'Sterilize tools with 10% bleach solution',
            'Wash hands with soap before handling plants',
            'Remove infected plants immediately',
            'Control weeds and insects'
        ],
        'severity': 'High',
        'organic_treatment': 'None - strict sanitation and resistant varieties only'
    },
    'Tomato___healthy': {
        'description': 'Healthy tomato plant with normal growth and fruit production.',
        'care_tips': [
            'Stake or cage plants for support',
            'Prune suckers for indeterminate varieties',
            'Maintain consistent soil moisture',
            'Fertilize with balanced NPK, avoid excess nitrogen',
            'Monitor for hornworms and fruit worms',
            'Harvest when fruits are fully colored'
        ],
        'next_inspection': '7 days',
        'severity': 'None'
    }
}

# Define class names in order (must match model output)
CLASS_NAMES = [
    'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
    'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew',
    'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy',
    'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot',
    'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy',
    'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy',
    'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch',
    'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight',
    'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite', 'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy'
]

print(f"✅ Loaded {len(DISEASE_KNOWLEDGE_BASE)} disease entries")
print(f"✅ Loaded {len(CLASS_NAMES)} class names")

✅ Loaded 38 disease entries
✅ Loaded 38 class names


In [4]:
# Step 4: Build the Enhanced CNN Model (Based on your research paper architecture)

def build_enhanced_model(input_shape=(128, 128, 3), num_classes=38):
    """
    Enhanced CNN architecture based on:
    "Automated Crop Disease Detection Using Convolutional Neural Networks"
    by Sarangdhar.

    Improvements:
    - Added Batch Normalization for faster convergence
    - Increased dropout for better regularization
    - Added residual-like connections
    """

    model = Sequential([
        # Block 1: 32 filters
        Conv2D(filters=32, kernel_size=3, padding='same', activation='relu',
               input_shape=input_shape, name='conv1_1'),
        BatchNormalization(),
        Conv2D(filters=32, kernel_size=3, activation='relu', name='conv1_2'),
        BatchNormalization(),
        MaxPool2D(pool_size=2, strides=2, name='pool1'),

        # Block 2: 64 filters
        Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', name='conv2_1'),
        BatchNormalization(),
        Conv2D(filters=64, kernel_size=3, activation='relu', name='conv2_2'),
        BatchNormalization(),
        MaxPool2D(pool_size=2, strides=2, name='pool2'),

        # Block 3: 128 filters
        Conv2D(filters=128, kernel_size=3, padding='same', activation='relu', name='conv3_1'),
        BatchNormalization(),
        Conv2D(filters=128, kernel_size=3, activation='relu', name='conv3_2'),
        BatchNormalization(),
        MaxPool2D(pool_size=2, strides=2, name='pool3'),

        # Block 4: 256 filters
        Conv2D(filters=256, kernel_size=3, padding='same', activation='relu', name='conv4_1'),
        BatchNormalization(),
        Conv2D(filters=256, kernel_size=3, activation='relu', name='conv4_2'),
        BatchNormalization(),
        MaxPool2D(pool_size=2, strides=2, name='pool4'),

        # Block 5: 512 filters
        Conv2D(filters=512, kernel_size=3, padding='same', activation='relu', name='conv5_1'),
        BatchNormalization(),
        Conv2D(filters=512, kernel_size=3, activation='relu', name='conv5_2'),
        BatchNormalization(),
        MaxPool2D(pool_size=2, strides=2, name='pool5'),

        # Classifier
        Dropout(0.25, name='dropout1'),
        Flatten(name='flatten'),
        Dense(units=1500, activation='relu', name='dense1'),
        BatchNormalization(),
        Dropout(0.4, name='dropout2'),
        Dense(units=num_classes, activation='softmax', name='output')
    ])

    # Compile with Adam optimizer (learning rate 0.0001 as per paper)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

# Create model instance
model = build_enhanced_model()
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1_1 (Conv2D)                │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv2D)                │ (None, 126, 126, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_1 (Conv2D)                │ (None, 63, 63, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 63, 63, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2_2 (Conv2D)                │ (None, 61, 61, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_1 (Conv2D)                │ (None, 30, 30, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 30, 30, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3_2 (Conv2D)                │ (None, 28, 28, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool3 (MaxPooling2D)            │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_1 (Conv2D)                │ (None, 14, 14, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 14, 14, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4_2 (Conv2D)                │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 12, 12, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool4 (MaxPooling2D)            │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv5_1 (Conv2D)                │ (None, 6, 6, 512)      │     1,180,16

 Total params: 7,856,698 (29.97 MB)

 Trainable params: 7,849,730 (29.94 MB)

 Non-trainable params: 6,968 (27.22 KB)

In [5]:
# Step 5: Data Preprocessing and Augmentation (Enhanced from paper)

def create_data_generators(train_dir='train', valid_dir='valid',
                          image_size=(128, 128), batch_size=32):
    """
    Create enhanced data generators with augmentation as described in the paper:
    - Image resizing to 128x128
    - Normalization
    - Data augmentation (rotation, flipping, brightness)
    """

    # Training data with augmentation
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )

    # Validation data - only rescaling
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
    )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    valid_generator = valid_datagen.flow_from_directory(
        valid_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, valid_generator

# For tf.data API (more efficient)
def create_tf_dataset(directory, image_size=(128, 128), batch_size=32,
                     shuffle=True, augment=False):
    """
    Create optimized tf.data pipeline as used in the original notebooks
    """

    dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='categorical',
        image_size=image_size,
        batch_size=batch_size,
        shuffle=shuffle,
        interpolation='bilinear'
    )

    # Normalize
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

    # Cache and prefetch for performance
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

print("✅ Data preprocessing functions defined")

✅ Data preprocessing functions defined


In [6]:
# Step 6: Model Training with Callbacks (Based on paper's 10-epoch training)

def train_model(model, train_data, valid_data, epochs=10, save_path='plant_disease_model.keras'):
    """
    Train the model with callbacks for checkpointing and early stopping
    Replicates the training from the research paper
    """

    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            save_path,
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7,
            verbose=1
        ),
        tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]

    history = model.fit(
        train_data,
        validation_data=valid_data,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )

    return history

# Visualization function for training history
def plot_training_history(history):
    """
    Plot training and validation metrics as shown in the paper
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Training', color='red')
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation', color='blue')
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Loss
    axes[0, 1].plot(history.history['loss'], label='Training', color='red')
    axes[0, 1].plot(history.history['val_loss'], label='Validation', color='blue')
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Precision
    if 'precision' in history.history:
        axes[1, 0].plot(history.history['precision'], label='Training', color='red')
        axes[1, 0].plot(history.history['val_precision'], label='Validation', color='blue')
        axes[1, 0].set_title('Precision', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Precision')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)

    # Recall
    if 'recall' in history.history:
        axes[1, 1].plot(history.history['recall'], label='Training', color='red')
        axes[1, 1].plot(history.history['val_recall'], label='Validation', color='blue')
        axes[1, 1].set_title('Recall', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Recall')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fig

print("✅ Training functions defined")

✅ Training functions defined


In [7]:
# Step 7: Grad-CAM Implementation for Explainability

def make_gradcam_heatmap(img_array, model, last_conv_layer_name='conv5_2', pred_index=None):
    """
    Generate Grad-CAM heatmap for model interpretability
    Shows which parts of the image the model focuses on
    """

    # Create model that maps input to activations of last conv layer and output predictions
    grad_model = tf.keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute gradient of top predicted class
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Gradient of output neuron with respect to feature map
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Vector of mean intensity of gradient over feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight feature maps by importance
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize between 0 and 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()

def superimpose_heatmap(img, heatmap, alpha=0.4):
    """
    Superimpose heatmap on original image
    """
    # Rescale heatmap to 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create image with RGB colorized heatmap
    jet_heatmap = tf.keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.utils.img_to_array(jet_heatmap)

    # Superimpose
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.utils.array_to_img(superimposed_img)

    return superimposed_img

print("✅ Grad-CAM functions defined")

✅ Grad-CAM functions defined


In [8]:
# Step 8 (DEFENSIVE VERSION): Check dependencies before defining class

# Check if required functions/classes exist
missing_dependencies = []

try:
    build_enhanced_model
except NameError:
    missing_dependencies.append("build_enhanced_model()")

try:
    CLASS_NAMES
except NameError:
    missing_dependencies.append("CLASS_NAMES")

try:
    DISEASE_KNOWLEDGE_BASE
except NameError:
    missing_dependencies.append("DISEASE_KNOWLEDGE_BASE")

try:
    make_gradcam_heatmap
except NameError:
    missing_dependencies.append("make_gradcam_heatmap()")

try:
    superimpose_heatmap
except NameError:
    missing_dependencies.append("superimpose_heatmap()")

if missing_dependencies:
    error_msg = f"""
    ❌ ERROR: Missing dependencies!

    The following required components are not defined:
    {', '.join(missing_dependencies)}

    📋 REQUIRED EXECUTION ORDER:
    1. Run Step 1: Install libraries
    2. Run Step 2: Import libraries
    3. Run Step 3: Disease knowledge base
    4. Run Step 4: Build enhanced model ⬅️ YOU ARE HERE
    5. Then run this cell (Step 8)

    Please run the previous cells in order, then re-run this cell.
    """
    raise RuntimeError(error_msg)

print("✅ All dependencies found! Proceeding with chatbot initialization...")

# Now define the class safely
class PlantDiseaseChatbot:
    """
    Advanced Plant Disease Detection Chatbot System
    Based on: "Automated Crop Disease Detection Using Convolutional Neural Networks"
    """

    def __init__(self, model_path=None):
        self.model = None
        self.model_path = model_path
        self.conversation_history = []
        self.confidence_threshold = 0.7
        self.image_size = (128, 128)

        # Build model architecture - NOW build_enhanced_model is guaranteed to exist
        self.model = build_enhanced_model()

        # Load weights if provided
        if model_path and os.path.exists(model_path):
            self.load_model(model_path)

        # Initialize NLP pipeline for advanced queries (optional)
        try:
            self.nlp = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")
            self.nlp_available = True
        except:
            self.nlp_available = False

        print("🌱 Plant Disease Chatbot initialized successfully!")

    def load_model(self, path):
        """Load trained model weights"""
        self.model = load_model(path)
        print(f"✅ Model loaded from {path}")

    def save_model(self, path='plant_disease_chatbot_model.keras'):
        """Save current model"""
        self.model.save(path)
        print(f"✅ Model saved to {path}")

    def preprocess_image(self, img_path):
        """Load and preprocess image for prediction"""
        img = image.load_img(img_path, target_size=self.image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Normalize
        return img_array, img

    def predict_disease(self, img_path):
        """Predict disease from image with confidence scores"""
        if self.model is None:
            return {"error": "Model not loaded. Please train or load a model first."}

        # Preprocess
        img_array, original_img = self.preprocess_image(img_path)

        # Predict
        predictions = self.model.predict(img_array, verbose=0)[0]

        # Get top 3 predictions
        top_indices = np.argsort(predictions)[-3:][::-1]
        top_predictions = []

        for idx in top_indices:
            class_name = CLASS_NAMES[idx]
            confidence = float(predictions[idx])
            disease_info = DISEASE_KNOWLEDGE_BASE.get(class_name, {})

            top_predictions.append({
                'class': class_name,
                'confidence': confidence,
                'disease_name': class_name.replace('___', ' - ').replace('_', ' '),
                'is_healthy': 'healthy' in class_name.lower(),
                'severity': disease_info.get('severity', 'Unknown'),
                'info': disease_info
            })

        # Determine if prediction is confident enough
        primary_prediction = top_predictions[0]
        is_confident = primary_prediction['confidence'] >= self.confidence_threshold

        # Generate Grad-CAM if confident
        heatmap = None
        if is_confident and not primary_prediction['is_healthy']:
            try:
                heatmap = make_gradcam_heatmap(img_array, self.model)
            except Exception as e:
                print(f"Grad-CAM failed: {e}")

        return {
            'primary_prediction': primary_prediction,
            'top_3_predictions': top_predictions,
            'is_confident': is_confident,
            'heatmap': heatmap,
            'original_image': original_img,
            'processed_array': img_array
        }

    def generate_response(self, prediction_result, user_query=None):
        """Generate natural language response based on prediction"""
        if 'error' in prediction_result:
            return prediction_result['error'], None

        pred = prediction_result['primary_prediction']
        info = pred['info']

        # Build response
        response_parts = []

        # Greeting and main diagnosis
        if pred['is_healthy']:
            response_parts.append(f"✅ **Good news!** Your plant appears to be **HEALTHY** ({pred['disease_name']}).")
            response_parts.append(f"\n**Confidence:** {pred['confidence']*100:.1f}%")

            if 'care_tips' in info:
                response_parts.append("\n### 🌱 Care Recommendations:")
                for tip in info['care_tips']:
                    response_parts.append(f"• {tip}")

            if 'next_inspection' in info:
                response_parts.append(f"\n**Recommended next inspection:** {info['next_inspection']}")

        else:
            response_parts.append(f"⚠️ **Disease Detected:** {pred['disease_name']}")
            response_parts.append(f"\n**Confidence:** {pred['confidence']*100:.1f}%")
            response_parts.append(f"**Severity Level:** {info.get('severity', 'Unknown')}")

            # Description
            if 'description' in info:
                response_parts.append(f"\n### 📋 Description:\n{info['description']}")

            # Cause
            if 'cause' in info:
                response_parts.append(f"\n### 🔬 Cause:\n{info['cause']}")

            # Symptoms
            if 'symptoms' in info:
                response_parts.append("\n### 🔍 Key Symptoms:")
                for symptom in info['symptoms']:
                    response_parts.append(f"• {symptom}")

            # Treatment
            if 'treatment' in info:
                response_parts.append("\n### 💊 Immediate Treatment:")
                for i, treatment in enumerate(info['treatment'][:5], 1):
                    response_parts.append(f"{i}. {treatment}")

            # Prevention
            if 'prevention' in info:
                response_parts.append("\n### 🛡️ Prevention Measures:")
                for prevention in info['prevention'][:3]:
                    response_parts.append(f"• {prevention}")

            # Organic options
            if 'organic_treatment' in info:
                response_parts.append(f"\n### 🌿 Organic Treatment Options:\n{info['organic_treatment']}")

        # Add alternative possibilities if low confidence
        if not prediction_result['is_confident']:
            response_parts.append("\n---")
            response_parts.append("⚠️ **Note:** Confidence is below threshold. Other possibilities:")
            for alt in prediction_result['top_3_predictions'][1:]:
                response_parts.append(f"• {alt['disease_name']} ({alt['confidence']*100:.1f}%)")

        # Add disclaimer
        response_parts.append("\n---")
        response_parts.append("**Disclaimer:** This AI prediction is for guidance only. For critical decisions, consult a local agricultural extension service or plant pathologist.")

        full_response = "\n".join(response_parts)

        # Store in conversation history
        self.conversation_history.append({
            'query': user_query or 'Image analysis',
            'prediction': pred['disease_name'],
            'confidence': pred['confidence'],
            'response': full_response
        })

        return full_response, prediction_result

    def answer_question(self, question, context=None):
        """Answer natural language questions about plant diseases"""
        question_lower = question.lower()

        # Common question patterns
        if any(word in question_lower for word in ['treat', 'cure', 'fix', 'medicine']):
            return "For treatment recommendations, please upload an image of the affected plant leaf, and I'll provide specific treatment options based on the detected disease."

        elif any(word in question_lower for word in ['prevent', 'avoid', 'stop']):
            return "Prevention is key! General tips: 1) Use certified disease-free seeds/transplants, 2) Rotate crops annually, 3) Maintain proper spacing for air circulation, 4) Avoid overhead irrigation, 5) Remove and destroy infected plant debris. For specific prevention, please share an image of your plant."

        elif any(word in question_lower for word in ['organic', 'natural', 'chemical-free']):
            return "Organic treatments vary by disease but commonly include: neem oil, copper fungicides, sulfur sprays, Bacillus subtilis products, insecticidal soaps, and biological controls. Upload an image for specific organic recommendations."

        elif any(word in question_lower for word in ['severity', 'dangerous', 'bad']):
            return "Disease severity ranges from None (healthy) to Critical. Critical diseases like Citrus Greening or Late Blight require immediate action. Upload an image to assess severity."

        elif any(word in question_lower for word in ['hello', 'hi', 'hey']):
            return "Hello! 🌱 I'm your Plant Disease Detection Assistant. I can help identify diseases from leaf images and provide treatment recommendations. Please upload a clear image of the affected plant leaf!"

        else:
            return "I can help you identify plant diseases and provide treatment recommendations. Please upload a clear image of the plant leaf showing symptoms, or ask me about specific diseases, treatments, or prevention methods."

    def generate_report(self):
        """Generate a text report of the session"""
        report = []
        report.append("# Plant Disease Detection Report")
        report.append(f"Generated by: Automated Crop Disease Detection System")
        report.append(f"Based on: Sarangdhar et al. (2024) - CNN-based Plant Disease Detection")
        report.append("=" * 50)
        report.append("")

        for i, entry in enumerate(self.conversation_history, 1):
            report.append(f"## Analysis {i}")
            report.append(f"**Query:** {entry['query']}")
            report.append(f"**Detected:** {entry['prediction']}")
            report.append(f"**Confidence:** {entry['confidence']*100:.1f}%")
            report.append("")
            report.append("**Response:**")
            report.append(entry['response'])
            report.append("-" * 50)
            report.append("")

        return "\n".join(report)

# Initialize chatbot instance globally
print("Initializing chatbot...")
chatbot = PlantDiseaseChatbot()
print(f"✅ Chatbot ready! Model has {chatbot.model.count_params():,} parameters")

✅ All dependencies found! Proceeding with chatbot initialization...
Initializing chatbot...


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/102 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

🌱 Plant Disease Chatbot initialized successfully!
✅ Chatbot ready! Model has 7,856,698 parameters


In [9]:
# Step 9: Create the Gradio Interface (ChatGPT-like UI)

def create_chatbot_interface():
    """
    Create an interactive Gradio interface for the Plant Disease Chatbot
    """

    # CSS for custom styling
    css = """
    .chatbot-container {
        max-height: 600px;
        overflow-y: auto;
    }
    .disease-card {
        border-left: 4px solid #e74c3c;
        padding-left: 15px;
        margin: 10px 0;
        background: #fdf2f2;
    }
    .healthy-card {
        border-left: 4px solid #27ae60;
        padding-left: 15px;
        margin: 10px 0;
        background: #f0fff4;
    }
    .confidence-high { color: #27ae60; font-weight: bold; }
    .confidence-medium { color: #f39c12; font-weight: bold; }
    .confidence-low { color: #e74c3c; font-weight: bold; }
    """

    # State management
    state = {
        'chatbot': chatbot,
        'history': []
    }

    def process_image(image_path, chat_history):
        """Process uploaded image and generate response"""
        if image_path is None:
            return chat_history, None, "Please upload an image first."

        # Get prediction
        result = state['chatbot'].predict_disease(image_path)

        # Generate response
        response, _ = state['chatbot'].generate_response(result)

        # Update chat history
        chat_history.append(("📸 Uploaded plant image", response))

        # Create visualization
        viz_fig = None
        if result.get('heatmap') is not None:
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Original image
            axes[0].imshow(result['original_image'])
            axes[0].set_title('Original Image')
            axes[0].axis('off')

            # Heatmap
            axes[1].imshow(result['heatmap'], cmap='jet')
            axes[1].set_title('Attention Heatmap\n(Where AI is looking)')
            axes[1].axis('off')

            # Superimposed
            super_img = superimpose_heatmap(
                image.img_to_array(result['original_image']),
                result['heatmap']
            )
            axes[2].imshow(super_img)
            axes[2].set_title('Combined Visualization')
            axes[2].axis('off')

            plt.tight_layout()
            viz_fig = fig

        # Confidence color coding
        conf = result['primary_prediction']['confidence']
        if conf >= 0.9:
            conf_class = "confidence-high"
        elif conf >= 0.7:
            conf_class = "confidence-medium"
        else:
            conf_class = "confidence-low"

        status = f"Analysis complete. Confidence: <span class='{conf_class}'>{conf*100:.1f}%</span>"

        return chat_history, viz_fig, status

    def process_text(message, chat_history):
        """Process text query"""
        if not message.strip():
            return chat_history, "Please enter a message."

        response = state['chatbot'].answer_question(message)
        chat_history.append((message, response))

        return chat_history, "Ready for next input."

    def clear_history():
        """Clear conversation history"""
        state['chatbot'].conversation_history = []
        return [], None, "History cleared. Ready for new analysis."

    def export_report():
        """Generate and return report"""
        report = state['chatbot'].generate_report()
        return report

    # Build interface
    with gr.Blocks(css=css, title="Plant Disease Detection Chatbot") as demo:
        gr.Markdown("""
        # 🌱 Plant Disease Detection Chatbot
        ### Powered by Convolutional Neural Networks
        **Based on Research:** *"Automated Crop Disease Detection Using Convolutional Neural Networks"*
        **Authors:** Kanishka Abhay Sarangdhar, Kanika Debbarma, Enjula Uchoi

        ---
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📤 Upload Image")
                image_input = gr.Image(type="filepath", label="Upload Plant Leaf Image")

                with gr.Row():
                    analyze_btn = gr.Button("🔍 Analyze Disease", variant="primary")
                    clear_btn = gr.Button("🗑️ Clear History")

                status_text = gr.Markdown("Status: Ready")

                gr.Markdown("### 💬 Or Ask a Question")
                text_input = gr.Textbox(
                    placeholder="Ask about treatments, prevention, or symptoms...",
                    label="Your Question"
                )
                ask_btn = gr.Button("💬 Ask", variant="secondary")

                gr.Markdown("### 📊 Export")
                export_btn = gr.Button("📄 Generate Report")
                report_output = gr.Textbox(label="Report", visible=False)

            with gr.Column(scale=2):
                chatbot_ui = gr.Chatbot(
                    label="Conversation",
                    height=500,
                    bubble_full_width=False
                )

                viz_output = gr.Plot(label="AI Visualization (Grad-CAM)")

        # Event handlers
        analyze_btn.click(
            fn=process_image,
            inputs=[image_input, chatbot_ui],
            outputs=[chatbot_ui, viz_output, status_text]
        )

        ask_btn.click(
            fn=process_text,
            inputs=[text_input, chatbot_ui],
            outputs=[chatbot_ui, status_text]
        )

        clear_btn.click(
            fn=clear_history,
            outputs=[chatbot_ui, viz_output, status_text]
        )

        export_btn.click(
            fn=export_report,
            outputs=gr.File(label="Download Report")
        )

        gr.Markdown("""
        ---
        ### 📝 Instructions:
        1. **Upload a clear image** of the affected plant leaf
        2. Click **"Analyze Disease"** to get AI diagnosis
        3. The AI will show **confidence scores** and **treatment recommendations**
        4. Use the **heatmap visualization** to see what the AI focuses on
        5. **Ask questions** about prevention, organic treatments, etc.
        6. **Export a report** for your records or to share with experts

        **Supported Crops:** Apple, Blueberry, Cherry, Corn, Grape, Orange, Peach, Pepper, Potato, Raspberry, Soybean, Squash, Strawberry, Tomato (38 disease classes total)

        **⚠️ Disclaimer:** This AI system is for educational and advisory purposes. Always consult local agricultural experts for critical decisions.
        """)

    return demo

# Create and launch interface
demo = create_chatbot_interface()

In [ ]:
# Step 10: Launch the Application

# For Google Colab - use this to expose the Gradio app
import threading
import time

def launch_app():
    """Launch the Gradio app with public URL"""
    demo.launch(
        share=True,  # Creates public URL
        debug=True,
        show_error=True
    )

# Launch in background thread so cell doesn't block
print("🚀 Launching Plant Disease Detection Chatbot...")
print("⏳ Please wait, generating public URL...")

# In Colab, run this to start
launch_app()

🚀 Launching Plant Disease Detection Chatbot...
⏳ Please wait, generating public URL...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5e308ddc968474d945.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Step 11: Complete Training Pipeline (Run this if you need to train from scratch)

def full_training_pipeline():
    """
    Complete training pipeline as described in the research paper
    """
    print("=" * 60)
    print("PLANT DISEASE DETECTION - TRAINING PIPELINE")
    print("Research Paper Implementation")
    print("=" * 60)

    # 1. Data Preparation
    print("\n[1/5] Preparing datasets...")
    # Note: Update these paths to your Google Drive or Colab paths
    train_dir = '/content/drive/MyDrive/plant_disease_dataset/train'
    valid_dir = '/content/drive/MyDrive/plant_disease_dataset/valid'

    # Use tf.data for efficiency
    train_ds = create_tf_dataset(train_dir, batch_size=32, shuffle=True)
    valid_ds = create_tf_dataset(valid_dir, batch_size=32, shuffle=False)

    print(f"✅ Training batches: {len(list(train_ds))}")
    print(f"✅ Validation batches: {len(list(valid_ds))}")

    # 2. Model Building
    print("\n[2/5] Building CNN model...")
    model = build_enhanced_model()

    # 3. Training
    print("\n[3/5] Training model (10 epochs as per paper)...")
    history = train_model(
        model,
        train_ds,
        valid_ds,
        epochs=10,
        save_path='trained_plant_disease_model.keras'
    )

    # 4. Evaluation
    print("\n[4/5] Evaluating model...")
    plot_training_history(history)

    # Print final metrics
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    print(f"\nFinal Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")

    # 5. Save metadata
    print("\n[5/5] Saving training metadata...")
    metadata = {
        'training_date': str(pd.Timestamp.now()),
        'epochs': 10,
        'final_train_acc': float(final_train_acc),
        'final_val_acc': float(final_val_acc),
        'model_architecture': 'Enhanced CNN with BatchNorm',
        'paper_reference': 'Sarangdhar et al. (2024)',
        'class_names': CLASS_NAMES
    }

    with open('training_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)

    print("\n✅ Training pipeline complete!")
    print("Files saved:")
    print("  - trained_plant_disease_model.keras")
    print("  - training_metadata.json")
    print("  - training_history.png")

    return model, history

# Uncomment to run training (requires dataset)
# model, history = full_training_pipeline()

In [ ]:
# Step 12: Quick Test Function (No training required - uses random weights for demo)

def demo_chatbot_without_training():
    """
    Run the chatbot interface without trained weights for UI demonstration
    In production, load your trained model
    """
    print("⚠️ Running in DEMO mode with untrained model")
    print("For accurate predictions, please train the model or load pre-trained weights")

    # The chatbot will still work but predictions will be random
    return create_chatbot_interface()

# Uncomment to run demo without training
# demo = demo_chatbot_without_training()
# demo.launch(share=True)

In [ ]:
# Step 13: Model Evaluation and Confusion Matrix (From your original notebook)

def evaluate_model(model, test_dir='valid'):
    """
    Comprehensive evaluation as shown in your research paper
    """
    from sklearn.metrics import classification_report, confusion_matrix
    import seaborn as sns

    # Load test data
    test_ds = tf.keras.utils.image_dataset_from_directory(
        test_dir,
        labels='inferred',
        label_mode='categorical',
        image_size=(128, 128),
        batch_size=32,
        shuffle=False
    )

    # Normalize
    normalization_layer = tf.keras.layers.Rescaling(1./255)
    test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

    # Predict
    y_pred = model.predict(test_ds)
    predicted_categories = tf.argmax(y_pred, axis=1)

    # True labels
    true_categories = tf.concat([y for x, y in test_ds], axis=0)
    Y_true = tf.argmax(true_categories, axis=1)

    # Classification report
    print("Classification Report:")
    print(classification_report(Y_true, predicted_categories, target_names=CLASS_NAMES))

    # Confusion Matrix
    cm = confusion_matrix(Y_true, predicted_categories)

    # Plot
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
    plt.xlabel('Predicted Class', fontsize=12)
    plt.ylabel('Actual Class', fontsize=12)
    plt.title('Plant Disease Prediction Confusion Matrix', fontsize=16)
    plt.xticks(rotation=90, fontsize=8)
    plt.yticks(rotation=0, fontsize=8)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

    return cm

# Run evaluation if model is trained
# cm = evaluate_model(model)

In [ ]:
# Step 14: Mount Google Drive and Setup (For Colab)

from google.colab import drive
drive.mount('/content/drive')

# Create directories for outputs
!mkdir -p /content/plant_disease_outputs
!mkdir -p /content/dataset

print("✅ Google Drive mounted")
print("✅ Output directories created")
print("\n📁 Please upload your dataset to:")
print("   /content/drive/MyDrive/plant_disease_dataset/")
print("   Structure: train/ and valid/ folders with class subdirectories")

# FINAL STEP: Complete Setup and Launch Instructions

"""
================================================================================
COMPLETE SETUP GUIDE FOR GOOGLE COLAB
================================================================================

1. DATASET PREPARATION:
   - Download PlantVillage dataset or use your custom dataset
   - Organize as:
     /content/drive/MyDrive/plant_disease_dataset/
       ├── train/
       │   ├── Apple___Apple_scab/
       │   ├── Apple___Black_rot/
       │   └── ... (38 classes)
       └── valid/
           ├── Apple___Apple_scab/
           └── ... (same 38 classes)

2. TRAINING (Run once):
   - Execute Step 11 (full_training_pipeline)
   - This trains for 10 epochs as per your research paper
   - Model achieves ~95% accuracy as reported

3. CHATBOT USAGE:
   - Execute Steps 1-10 to initialize everything
   - Run Step 10 to launch the Gradio interface
   - Share the public URL with users

4. FEATURES INCLUDED:
   ✅ CNN-based disease detection (128x128 input)
   ✅ 38 disease classes from research paper
   ✅ Confidence scoring with threshold (70%)
   ✅ Grad-CAM visualizations
   ✅ Natural language Q&A
   ✅ Treatment recommendations
   ✅ Organic treatment options
   ✅ Prevention guidelines
   ✅ Multi-turn conversation history
   ✅ Report generation
   ✅ Voice output (optional)

5. RESEARCH PAPER INTEGRATION:
   - All 38 classes from your paper included
   - Architecture matches paper specifications
   - Training parameters: Adam, lr=0.0001, 10 epochs
   - Batch normalization added for enhancement

6. DEPLOYMENT OPTIONS:
   - Option A: Run in Colab with share=True (temporary URL)
   - Option B: Download model and deploy to Hugging Face Spaces
   - Option C: Export to TensorFlow Lite for mobile

================================================================================
"""

print(setup_guide)